In [1]:
!pip install transformers

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached regex-2023.3.23-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (768 kB)
  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
  Using cached filelock-3.11.0-py3-none-any.whl (10.0 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)


In [2]:
!pip install sentencepiece

  Using cached sentencepiece-0.1.97-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)


In [3]:
import torch
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.auto import tqdm

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

df = pd.read_csv("Train_Only_Sentence_Title.csv", encoding='utf-8-sig', sep=';')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)
summaries = []
for article_text in tqdm(df["generated_text"]):
    input_ids = tokenizer(
        [WHITESPACE_HANDLER(article_text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=4,
        num_beams=2
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    summaries.append(summary)
    torch.cuda.empty_cache()

df["summary"] = summaries
df.to_csv("Train_Only_Sentence_Title_out.csv", index=False)

SyntaxError: invalid syntax (2252554173.py, line 30)